# 1- Selection of the initial dataset

In [ ]:
#Note:It was intended as much as possible clearing ambiguities in the following recapitulation of the codes used. 
#However the codes also depend on the directory in which the code is excuted
#All the files mentioned are present in the supplimentary materials



#Through rscb PDB, 167 sequences generated in the (rcsb_pdb_custom_report_20210501073759.csv) file
#Printing only the wanted columns:
awk -F "," '{print $(NF-1),$5,$3,$4}'  rcsb_pdb_custom_report_20210501073759.csv | tr -d '"' |tail -n +2 >clean_pdb.txt

#cleaning the file from the potentially co-crystallized sequences:
awk '{if ($4>=40 && $4<=80) print $0}' clean_pdb.txt >clean_pdb_len_40_80.txt

#Now, making another search to match the info from the first search with one to minimize any possible error probability

# File (data123.csv) is the result of using 3TGI chain I against PDB:
awk -F ',' '{print toupper($2),$3,$4}' data123.csv |tail -n +2 >3tgi_aln.list

# Getting the columns that we need in the second list and removing the top row, and make the entry id/chain uppercase
awk -F ',' '{print toupper($2),$3,$4}' data123.csv |tail -n +2 >3tgi_aln.list

# getting the 2 lists to be arranged so that only one column in both:
awk '{print $1$2}' clean_pdb_len_40_80.txt  |sort -u >list1

awk '{print $1}' 3tgi_aln.list  |sort -u >list2

#To create from clean_pdb_len_40_80.txt to create a file that generates an entry id and the chain concatenated and the sequence, generating a fasta file.
awk '{print $1$2, $3}' clean_pdb_len_40_80.txt  >clean_pdb.seq

#then using the python code (compare.py) to get the common sequences between the following 2 files:
python compare.py clean_pdb.seq list2 |awk '{print ">"$1;print$2}' >comm_seq.fasta 


#Now we are able to cluster them based on sequence similarity using blastclust or using cd-hit platform
#In cd-hit, we used 90 percent as a cutoff, and a minimal alignment coverage for the shorter sequence of 80 percent.
#Clustering them into 23 clusters, obtaining the sorted version and cleaning the file using the command:
awk '{if (substr($0,1,1)==">") {print "" } else {printf "%s ",substr($3,2,5)}}' cd-hit.cluster |tail -n +2 >cdhit-seq.list

#it was also done with Blastclust, resulting in 25 clusters in the file 
# Going with the output of (cd-hit.cluster), doing a for loop for the following code to obtain 23 SEPARATE clusters:
sed -n %ip cdhit-seq.list |awk '{split($0,a," "); for (j=1;j<=length(a);j++) {print substr(a[j],1,4)}}' |sort -u >cluster$i

#then another for loop to get the separate seed files:
Python compare.py resolu.idx cluster$i |sort -nk3 |head -n 1 | awk '{print substr($1,1,4)}' >seed$i

#then using the "cat" command to concatenate the 23 seed represntatives, resulting in the file (list_pdb.txt), 
#then sorting it and adding the chain, Note:(file seedcombination is a file where the wanted chain is added manually)

awk '{print substr($1,1,4)}' ../seedcombination.txt |sort -u >list_pdb.txt

# 2- Structural Alignment Generation

In [ ]:
#Now we got a list of identifiers, we want to build a multiple structural alignment based on them.
#Using a bash code, we will get the sequences of the 23 identifiers
#Creating a chain directory 

mkdir chains

#then getting whole pdb file of each one of the 23:

for i in `cat list_pdb.txt`
> do
> wget -q https://files.rcsb.org/view/$i.pdb

> done

#To space them out:

awk '{print substr($1,1,4),substr($1,5,1)}' ../seedcombination.txt |sort -u >list_chain.txt

#Using the (selch.sh) shell script to choose the chains:

awk '{print "./selch.sh",$1".pdb",$2" >chains/"$1$2".pdb"}' list_chain.txt >run.sh

#Then to generate a list of pdbfiles containing all the chains of the protein we are trying to align

/bin/bash run.sh

#Generating the zipped folder to analyze it using Mtmalign to do multiple structure alignment

tar -czvf chains.tar.gz chains/

#Removing a sequence of them to make the alignment more compact

rm 1D0DA.pdb

#Compress again, and then download the output

wget https://yanglab.nankai.edu.cn/mTM-align/output/mTM016214/seq.fasta -O tm-ali.fasta

#Cleaning the file

awk '{if (substr($0,1,1)==">") {print "\n"$0} else {printf "%s",$0}}' tm-ali.fasta  |tail -n +2 >tm-ali.clean

#Using also pdbe to make another alignment (as an extra check) is available (fastapdbefoldMSA.seq).
#Cleaning the file, removing extra rows, cutting the gapped edges:

awk '{if (substr($0,1,1)==">") {printf "\n%s ",$0} else {printf "%s",$0}}' tm-ali.fasta |awk '{print substr($1,1,6); print substr($2,28,61)}' |tail -n +3 >bpti_kunitz.ali





# 3- HMM Generation using HMMER

In [ ]:
hmmbuild bpti_kunitz.hmm bpti_kunitz.ali

# 4- Model Testing


In [ ]:
#Using the following codes in Uniprot to get the sets of positives and negatives respectively
#Positives:

database:(type:pfam pf00014) length:[40 TO *] AND reviewed:yes

#Negatives:

NOT database:(type:pfam pf00014) length:[40 TO *] AND reviewed:yes

#Unzipping the 2 files

zcat uniprot-database\ \(type\ pfam+pf00014\)+length\ \[40+TO+\ \]-filtered-rev--.list.gz  >positives.txt

zcat uniprot-NOT+database\ \(type\ pfam+pf00014\)+length\ \[40+TO+\ \]+reviewed--.fasta.gz >negatives.txt

#Getting just the list of idenitifiers

cat positives.fasta |awk '{if (substr($0,1,1)==">") {split($0,a,"|"); print a[2]}}' |less >positives.id

cat negatives.fasta |awk '{if (substr($0,1,1)==">") {split($0,a,"|"); print a[2]}}' |less >negatives.id


#Now, we need to sort them and then to select the first half and the other half
# 1- Sorting them randomly:
sort -R negatives.id >negatives.rids
sort -R positives.id >positives.rids


#2- Cut the halves to four halves:
#  i-positives:
head -n 178 positives.rids >set_pos1.ids
tail -n +179 pos	itives.rids >set_pos2.ids

#   ii-negatives:
head -n 277521 negatives.rids >set_neg1.ids
tail -n +277522 negatives.rids >set_neg2.ids

#Now we need to generate the fasta file that are needed for running the program, 

#we should write a very basic script that takes an input a list of identifiers and a fasta file and extracts fasta file from the file that you provided as an input.

python3 select-seqs.py pdb/set_pos1.ids positives.fasta >set_pos1.fasta
python3 select-seqs.py pdb/set_pos2.ids positives.fasta >set_pos2.fasta

python3 select-seqs.py pdb/set_neg1.ids negatives.fasta >set_neg1.fasta
python3 select-seqs.py pdb/set_neg2.ids negatives.fasta >set_neg2.fasta

#We also will disactivate the choice to discard the sequences that 
#do not have a very high seed match, and to get a better calculation for the match between the model and the sequence: --max
#-Z <x>        : set # of comparisons done, for E-value calculation

hmmsearch -Z 1 --noali --max --tblout set_neg1.out bpti_kunitz.hmm set_neg1.fasta

hmmsearch -Z 1 --noali --max --tblout set_neg2.out bpti_kunitz.hmm set_neg2.fasta

hmmsearch -Z 1 --noali --max --tblout set_pos1.out bpti_kunitz.hmm set_pos1.fasta

hmmsearch -Z 1 --noali --max --tblout set_pos2.out bpti_kunitz.hmm set_pos2.fasta

#Not all queries will produce hits, so calculating the hits:

grep -v "^#" set_pos1.out |awk '{print $1,$8,1}' >set_pos1.res
grep -v "^#" set_pos2.out |awk '{print $1,$8,1}' >set_pos2.res

grep -v "^#" set_neg1.out |awk '{print $1,$8,0}' >set_neg1.res
grep -v "^#" set_neg2.out |awk '{print $1,$8,0}' >set_neg2.res

#The process of recovering the missing sequences from the negatives

comm -23 <( sort set_neg1.ids) <(awk '{print $1}' set_neg1.res |sort) | awk '{print $1,10,0}'   >set_neg1.add
comm -23 <( sort set_neg2.ids) <(awk '{print $1}' set_neg2.res |sort) | awk '{print $1,10,0}'   >set_neg2.add

#Setting the complete set1 and set2

cat set_neg1.res set_neg1.add set_pos1.res >set_all1.res
cat set_neg2.res set_neg2.add set_pos2.res >set_all2.res

#Getting the threshold of set#1 using a python code that calculates the performance (in the zipped file)
python performance.py set_all1.res |sort -nrk 6

#Applying optimal threshold of set1 to set2
python performance.py set_all2.res 1e-05 

#Making a total set
awk '{p=0; if ($2<1e-5) {p=1}; print $1,p,$3}' set_all2.res > set_all.res
awk '{p=0; if ($2<1e-11) {p=1}; print $1,p,$3}' set_all1.res >> set_all.res

#getting the total ACC and MCC using a modified performance code 

python performance_all.py set_all.res 0.5

#Calculating false positives:
awk '{if ($2==1 && $3==0) print $1":false positives"}' set_all.res

#Calculating false negatives:
awk '{if ($2==0 && $3==1) print $1":false negative"}' set_all.res

#running the model against the misclassified sequences

#1- false negatives:
hmmsearch bpti_kunitz.hmm D3GGZ8.fasta |less
hmmsearch bpti_kunitz.hmm P86963.fasta |less
hmmsearch bpti_kunitz.hmm Q11101.fasta |less
hmmsearch bpti_kunitz.hmm O62247.fasta |less

#2- false postives:
hmmsearch bpti_kunitz.hmm P56409.fasta |less
hmmsearch bpti_kunitz.hmm P40500.fasta |less

